
# MLflow Basics: Tracking, Logging, and UI Exploration

In this notebook, you'll learn about key MLflow components including tracking, registry, and models. You'll explore how to track experiments, log parameters, metrics, and models, and use the MLflow UI for visualization and comparison.

**Tasks:**
- Train a simple regression model
- Log parameters, metrics, and the trained model with MLflow
- View experiment results in the MLflow UI
- Compare multiple runs to evaluate model performance

### Task-1  Train a simple regression model

In [0]:
# load spark table
from pyspark.sql import functions as F

events = spark.table("workspace.default.silver_ecommerce_events_event_type_part")


In [0]:
#convert data types
from pyspark.sql import functions as F

events = events.withColumn("product_id", F.col("product_id").cast("long")) \
               .withColumn("user_id", F.col("user_id").cast("long")) \
               .withColumn("price", F.col("price").cast("double")) \
               .withColumn("category_id", F.col("category_id").cast("long")) \
               .withColumn("event_time", F.col("event_time").cast("timestamp")) \
               .withColumn("event_date", F.to_date("event_time"))

In [0]:
events = events.withColumn(
    "revenue",
    F.when(F.col("event_type") == "purchase", F.col("price")).otherwise(0.0)
)
display(events.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,revenue
2019-10-26T07:55:55.000Z,view,1005118,2053013555631882655,electronics.smartphone,apple,997.44,552074397,5b166a28-b867-466e-9530-9031f26a42a0,2019-10-26,0.0
2019-10-26T07:55:55.000Z,view,1005073,2053013555631882655,electronics.smartphone,samsung,1135.24,515841537,43bac787-8766-4e86-aaa0-d6bbc5167887,2019-10-26,0.0
2019-10-26T07:55:55.000Z,view,10800217,2053013554994348409,null,redmond,54.03,515183873,395a5f66-e1ce-4c90-9afe-af9fbc1911df,2019-10-26,0.0
2019-10-26T07:55:57.000Z,view,1005115,2053013555631882655,electronics.smartphone,apple,984.25,535130435,45fa931d-294f-4fd6-94c0-62ce3f814cde,2019-10-26,0.0
2019-10-26T07:55:57.000Z,view,1005098,2053013555631882655,electronics.smartphone,samsung,140.73,514945316,7962b6b0-36b0-49a0-9374-0e0deab2df62,2019-10-26,0.0
2019-10-26T07:55:58.000Z,view,1005105,2053013555631882655,electronics.smartphone,apple,1375.82,555500241,ba01bd46-6682-42ea-b516-98eca4f15c0a,2019-10-26,0.0
2019-10-26T07:55:58.000Z,view,1005100,2053013555631882655,electronics.smartphone,samsung,140.7,564292459,0504f698-b481-4478-bf53-9f07703faf55,2019-10-26,0.0
2019-10-26T07:55:59.000Z,view,1005111,2053013555631882655,electronics.smartphone,apple,1234.01,537050418,134c006f-79e7-434b-a779-304716aaad21,2019-10-26,0.0
2019-10-26T07:55:59.000Z,view,1005150,2053013555631882655,electronics.smartphone,xiaomi,412.89,544561099,bdf5f1bc-467f-4ec8-ba7d-f558ff18a885,2019-10-26,0.0
2019-10-26T07:55:59.000Z,view,1005252,2053013555631882655,electronics.smartphone,oppo,295.73,564292405,ba88fd6e-8e9a-46b1-8d85-38df3e73dc33,2019-10-26,0.0


In [0]:
#select reuquired columns and convert spark table to pandas dataframe to build models
selected_df = events.select("product_id", "user_id", "price")
display(selected_df.limit(10))

pandas_df = selected_df.toPandas()

product_id,user_id,price
1004858,522850155,131.53
1004864,515027636,115.81
1004767,550404843,245.28
1004768,557311182,245.26
1004838,564297430,141.29
1004992,512430659,231.64
1004766,563561200,244.32
1004741,540014734,189.97
1004870,547109676,281.11
1004836,533886616,223.76


In [0]:
#select required columns for features and label to build models, feature scale the columns, split data into train and test sets and train a linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = pandas_df[["product_id" , "user_id"]]
y = pandas_df["price"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [0]:
#Evaluate model performance
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

Mean Squared Error: 121327.31817004352
R2 Score: 0.054406666660744674


In [0]:
#inspect model coefficients
print("Model coefficients:", model.coef_)
print("Model intercept:", model.intercept_)

Model coefficients: [-83.67488275  -1.51953436]
Model intercept: 290.3184957051498


### Task-2 Log parameters, metrics, and the trained model with MLflow

Goal:
Track what is trained, how it performed, and store the model.

Purpose:
Demonstrate core MLflow tracking skills for reproducible machine learning.

In [0]:
import mlflow
import mlflow.sklearn

#### Log Experiment with MLflow

Purpose:
Wrap training and evaluation inside an MLflow run to log parameters, metrics, and the trained model for reproducibility.

In [0]:
from sklearn.metrics import mean_squared_error, r2_score

with mlflow.start_run(run_name="linear_regression_v1"):
    # Log parameters
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("features", "product_id, user_id")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)

    # Train model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2_score", r2)

    # Log model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        input_example=X_train[:5]
    )

print(f"Mean Squared Error logged: {mse:.4f}")
print(f"R² Score logged: {r2:.4f}")

Mean Squared Error logged: 121327.3182
R² Score logged: 0.0544


MLflow Run Results
Parameters, metrics, and model artifacts are logged and can be viewed in the MLflow UI.

R² Score for this run is logged for comparison.

### Task-3 View experiment results in the MLflow UI

Visualize the experiment, metrics, parameters, and model in the MLflow tracking UI for transparency and auditability.

### Task-4 Compare multiple runs to evaluate model performance

Understand how MLflow helps to choose the best model by comparing multiple experiment runs and their metrics.

In [0]:
# Wrap training and evaluation inside an MLflow run to log parameters, metrics, and the trained model without user_id feature

with mlflow.start_run(run_name="linear_regression_no_user_id"):
    # Log parameters
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("features", "product_id")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)

    # Prepare data without user_id
    X_no_user = pandas_df[["product_id"]]
    scaler_no_user = StandardScaler()
    X_no_user_scaled = scaler_no_user.fit_transform(X_no_user)
    X_train_no_user, X_test_no_user, y_train_no_user, y_test_no_user = train_test_split(
        X_no_user_scaled, y, test_size=0.2, random_state=42
    )

    # Train model
    model_no_user = LinearRegression()
    model_no_user.fit(X_train_no_user, y_train_no_user)

    # Predict and evaluate
    y_pred_no_user = model_no_user.predict(X_test_no_user)
    mse_no_user = mean_squared_error(y_test_no_user, y_pred_no_user)
    r2_no_user = r2_score(y_test_no_user, y_pred_no_user)

    # Log metrics
    mlflow.log_metric("mse", mse_no_user)
    mlflow.log_metric("r2_score", r2_no_user)

    # Log model
    mlflow.sklearn.log_model(
        sk_model=model_no_user,
        artifact_path="model",
        input_example=X_train_no_user[:5]
    )
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

Mean Squared Error: 121327.31817004352
R2 Score: 0.054406666660744674


### Conclusion

**Comparison of R² Scores**

Product_id + User_id model: R² ≈ 0.0544

Product_id only model: R² ≈ 0.054406666660

Adding User_id as a feature did not provide a significant boost in predictive accuracy.

**Model Comparison**

linear_regression_v1 (features: Product_id + User_id)
 R² ≈ 0.0544

linear_regression_no_user_id (feature: Product_id only)
 R² ≈ 0.054406666660

**Key Insight**

Adding User_id as a feature did not improve predictive performance for price.